In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#cnn
<img src="data/mnist.png">

In [60]:
# show plot first run
%matplotlib inline 
# test autocompletion with tab or tab+shift
%config IPCompleter.greedy=True 


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        #print(size)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [76]:
params = list(net.parameters())
print(len(params))
for i in range(len(params)):
    print('i:',i,params[i].size())  # conv1's .weight

#print(params[1].size())  # conv1's .weight
#print(params[0])

10
i: 0 torch.Size([6, 1, 3, 3])
i: 1 torch.Size([6])
i: 2 torch.Size([16, 6, 3, 3])
i: 3 torch.Size([16])
i: 4 torch.Size([120, 576])
i: 5 torch.Size([120])
i: 6 torch.Size([84, 120])
i: 7 torch.Size([84])
i: 8 torch.Size([10, 84])
i: 9 torch.Size([10])


In [62]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0292, -0.0745, -0.1197, -0.1122, -0.1540, -0.0466, -0.0395, -0.0627,
          0.0273, -0.0633]], grad_fn=<AddmmBackward>)


In [63]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [68]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(2.3199, grad_fn=<MseLossBackward>)


In [69]:
#loss.backward()

In [71]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [72]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0206,  0.0191,  0.0041,  0.0009, -0.0076, -0.0263])


In [75]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update